# **Twitter TD 3**

Dates: 2020



## Initial Setup

- **Run "Setup" below first.**

    - This will load libraries and download some resources that we'll use throughout the tutorial.

    - You will see a message reading "Done with setup!" when this process completes.


In [1]:
#@title Setup (click the "run" button to the left) {display-mode: "form"}

## Setup ##

# imports

# built-in Python libraries
# -------------------------

! pip install peony-twitter[all]
! pip install peony-twitter[all]
! pip show tornado
! pip install nest_asyncio 
import nest_asyncio
import time
import json
import datetime
from peony import EventStream, PeonyClient, event_handler, events
import asyncio
import random
import sys
from pathlib import Path
nest_asyncio.apply()


print("Done with setup!")
print("If you'd like, you can click the (X) button to the left to clear this output.")

# generate graph from twitter json
def create_graph(data):
    all_data = dict(data)
    edge_list = []
    user_screen_name = all_data['user']['screen_name']
    timestamp_ms = all_data['timestamp_ms']
    if len(all_data['entities']['hashtags']) > 0:
        for index, i in enumerate(all_data['entities']['hashtags']):
            edge_list.append(("@"+user_screen_name, "#"+all_data['entities']['hashtags'][index]['text'], timestamp_ms))
    if len(all_data['entities']['user_mentions']) > 0:
        for index, i in enumerate(all_data['entities']['user_mentions']):
            edge_list.append(("@"+user_screen_name, "@"+all_data['entities']['user_mentions'][index]['screen_name'], timestamp_ms))
    return edge_list
  
# compute components from reservoir of edges
def components(edges):
  global d
  d={}
  n=0
  for e in edges:
     a =e [0]
     b = e[1]
     if a in d.keys():
        d[a].add(b)
     else:
      d[a]=set([b])
     if b in d.keys():
      d[b].add(a)
     else:
      d[b]=set([a])

  #print ("Dict=", d, len(d))
  n=len(d)
  m=0
  #Breadth-first search from first point, first component
  a=list(d.keys())[0]

  #dc keeps b:i  if i is the length of the shortest path from a to b in the first component
  dc={}
  dc[a]=0
  for b in d[a]: 
    if b in dc.keys(): pass
    else: dc[b]=1

  #Initialize S ans S1, Start iterating
  S=d[a]
  comp=[]
  S1=set([a])
  S=S.union(S1)
  #print("S=",S)
  while S > S1:
      S1=S
      for u in S:
         S=S.union(d[u])
         for v in d[u]:
           if v in dc.keys():
              dc[v]=min(dc[v],dc[u]+1)
           else:
            dc[v]=dc[u]+1
  for u in S:
       m=m+len(d[u])

  comp.append((len(S),int(m/2),list(S)))
    
  #print("Component",comp)

  ST=S
    
  #print("ST=",ST)

  #The other components: origin must be outside ST, same treatment
  i=1
  while i<len(d):
   m=0
   while  list(d.keys())[i] not in ST:
    a=list(d.keys())[i]
    S=d[a]
    S1=set([a])
    S=S.union(S1)
    while S > S1:
      S1=S
      for u in S:
         S=S.union(d[u])
    for u in S:
       m=m+len(d[u])
    comp.append((len(S),int(m/2),list(S)))
    ST=ST.union(S)  
   i+=1     
  return comp


def comp_edges(comp1):
 compedges=[]
 j0=0
 while j0<len(comp1):
   l1 = comp1[j0][2]
   j=0
   cp=[]
   while j <len(l1):
     a=l1[j]
     j1=0
     while j1< len(d[l1[j]]):
      f=(a,list(d[l1[j]])[j1])
      cp.append(f)
      j1 +=1
     j +=1
   j0 +=1
   compedges.append(cp)
 return compedges

# write edges into file

def write_edge_reservoir(i, time_export):
    f = open("sample_data_step_reservoir_edges.csv", "a")
    # f.write("Source, Destination \n")
    for j in i:
      f.write("%s, %s\n" %j)
    f.close()
    return True

  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached aiodns-3.0.0-py3-none-any.whl (5.0 kB)
  Using cached aiofiles-0.8.0-py3-none-any.whl (13 kB)
  Using cached pycares-4.1.2-cp310-cp310-win_amd64.whl (83 kB)
  Using cached cffi-1.15.0-cp310-cp310-win_amd64.whl (180 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Using legacy 'setup.py install' for cchardet, since package 'wheel' is not installed.
  Running setup.py install for cchardet: started
  Running setup.py install for cchardet: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Running setup.py install for cchardet did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running install
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.10
      creating build\lib.win-amd64-3.10\cchardet
      copying src\cchardet\version.py -> build\lib.win-amd64-3.10\cchardet
      copying src\cchardet\__init__.py -> build\lib.win-amd64-3.10\cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cchardet

note: This is an issue with the package mentioned above, not pip.
hint: See ab

  Using cached aiofiles-0.8.0-py3-none-any.whl (13 kB)
  Using cached aiodns-3.0.0-py3-none-any.whl (5.0 kB)
  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pycares-4.1.2-cp310-cp310-win_amd64.whl (83 kB)
  Using cached cffi-1.15.0-cp310-cp310-win_amd64.whl (180 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Using legacy 'setup.py install' for cchardet, since package 'wheel' is not installed.
  Running setup.py install for cchardet: started
  Running setup.py install for cchardet: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Running setup.py install for cchardet did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running install
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.10
      creating build\lib.win-amd64-3.10\cchardet
      copying src\cchardet\version.py -> build\lib.win-amd64-3.10\cchardet
      copying src\cchardet\__init__.py -> build\lib.win-amd64-3.10\cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cchardet

note: This is an issue with the package mentioned above, not pip.
hint: See ab

Name: tornado
Version: 6.1
Summary: Tornado is a Python web framework and asynchronous networking library, originally developed at FriendFeed.
Home-page: http://www.tornadoweb.org/
Author: Facebook
Author-email: python-tornado@googlegroups.com
License: http://www.apache.org/licenses/LICENSE-2.0
Location: c:\users\sakth\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: ipykernel, jupyter-client
Done with setup!
If you'd like, you can click the (X) button to the left to clear this output.


## The Twitter API 

- Twitter is also known for being an abundant source of publc text data (perhaps even more so than Reddit).
- Twitter provides several types of API that can be used to collect anything from tweets to user descriptions to follower networks.
    - You can [read all about it here](https://developer.twitter.com/).
- For this tutorial, we'll look at using the [streaming API](https://developer.twitter.com/en/docs/tutorials/consuming-streaming-data.html), which allows us to retreive tweets that contain specific words, phrases, and hashtags.
- In the slides, we talked about how to setup a Twitter App and get a API keys.
    - You should add your own keys below and then run the code block to set your keys:
    
#### [Tutorial link to create an Twitter app](https://botwiki.org/resource/tutorial/how-to-create-a-twitter-app/) or https://www.youtube.com/watch?v=aB1x7ZsbbaQ&feature=youtu.be

In [2]:
# Go to http://dev.twitter.com and create an app. 
# The consumer key and secret will be generated for you after
consumer_key=""
consumer_secret=""

# After the step above, you will be redirected to your app's page.
# Create an access token under the the "Your access token" section
access_token=""
access_token_secret=""

- Do not share your credentials with anyone!
    - You shouldn't hardcode your API keys in code (like above) if you are going to save the file anywhere that is visible to others (like commiting the file to github).
        - You can read more about securing your API keys [here](https://developer.twitter.com/en/docs/basics/authentication/guides/securing-keys-and-tokens).
     - So, if you plan to save this file in any way, make sure to remove your API keys first.
     - If you think your keys have been compromized, you can regenerate them.
        - [Apps](https://developer.twitter.com/en/apps) -> Keys and Tokens -> Regenerate

## Capture Data

![Step reservoir sampling](https://github.com/GuillaumeVIMONT/TD_UP2_new/blob/master/static/img/continuous_sampling.png?raw=true)

Before start Twitter capture we need some informations :

* $k$ is the size of the reservoir \\
* $tau$ is the lenght of the window \\
* $lambda$ is the lenght of a strate \\
* $tracking$ is the keywords to filter Twitter stream \\
* $threshold$ is the threshold for the connected component export \\
* $timeout$ is the duration of the capture

In [ ]:
#@title Capture Twitter streaming (double-click to view) {display-mode: "form"}

k = 200 #@param {type:"number"}
tau = 3 #@param {type:"number"}
lamb = 1 #@param {type:"number"}
tracking = "CNN" #@param {type:"string"}
threshold = 5 #@param {type:"slider", min:1, max:20, step:1}
timeout = 5 #@param {type:"slider", min:1, max:120, step:1}
timeout = int(timeout)*60000
start = int(round(time.time() * 1000))

global L, sample_window_stream, global_counter, time_counter, interval_counter
L=0
sample_window_stream = []
global_counter = 0
interval_counter = 0
time_counter = (time.time()*1000)+60000


# k is the size of the reservoir (number of edges into the reservoir)


# tau is the lenght of the window
tau =  tau*60000

# lamb is the lenght of strate floato the window
lamb = lamb*60000

# rate is the number of strates per window
rate = tau/lamb

# M contain steps floato the window, for each strates we have a edges counter
M = [0] * int(rate)

# init it's a default parameters to configure somes variables when the reservoir strart
init = 0

reservoir = []


def step_reservoir_sampling(edge):
    global init, w_i, w_1, M, window_counter, lamb, t_i, dict_test, tau, t
    if init == 0:
        # initialise w_i
        w_1 = int(edge[2])+tau
        w_i = int(edge[2])+tau
        t_i = int(edge[2])+lamb
        # window counter
        window_counter = 1
        # terminate initialisation
        init+=1
    # check if edge is indide w_i
    if int(edge[2]) < t_i:
        M[-1]+=1
    if int(edge[2]) >= t_i:
        print(M)
        if int(edge[2]) >= w_1:
          comp = components(reservoir)
          comp1 = sorted(comp, reverse=True)
          xx = comp_edges(comp1)
          now = datetime.datetime.now()
          time_export = now.strftime("%Y_%m_%d_%Hh%M")
          f = open("sample_data_step_reservoir_edges.csv", "a")
          f.write("Source, Target\n")
          f.close()
          index = 0
          for i in comp1:
              if int(i[0]) >= int(threshold):
                  write_edge_reservoir(xx[index], time_export)
                  index+=1
              else:
                  break
        while len(reservoir) > 0 and int(reservoir[0][2]) < w_i-tau:
            # while the previous condition is satisfy do
            del reservoir[0]
        window_counter +=1
        del M[0]
        M.append(1)
        t_i += lamb
        w_i += lamb
    if sum(M) < k:
        reservoir.append(edge)
    else:
        j = random.randint(0, sum(M))
        if j < k:
            if len(reservoir) < k:
                reservoir.append(edge)
            else:
                del reservoir[j]
                reservoir.append(edge)

    return reservoir


# Get date
now = datetime.datetime.now()
def ask_exit():
    for task in asyncio.all_tasks():
        task.cancel()
async def consume(queue):
    while True:
        # wait for an item from the producer
        item = await queue.get()
        twitter_edges_graph = create_graph(item)
        queue.task_done()
        global time_counter, global_counter, interval_counter
        if len(twitter_edges_graph) > 0:
            # process the item
            # print('consuming {}...'.format(twitter_edges_graph))
            # Notify the queue that the item has been processed
            for edge in twitter_edges_graph:
                if tracking[0] in edge[1].lower():
                    pass
                else:
                    global_counter+=1
                    interval_counter+=1
                    window_reservoir_sampling = step_reservoir_sampling(edge)
        if time.time()*1000 > start + timeout:
            print('Capture terminated')
            ask_exit()



class Client(PeonyClient):
    pass

# every class inheriting from `PeonyClient` or `BasePeonyClient` has
# an event_stream function that can be used on an `EventStream`
@Client.event_stream
class UserStream(EventStream):

    def stream_request(self):
        """
            The stream_request method returns the request
            that will be used by the stream
        """
        return self.stream.statuses.filter.post(track=tracking)


    # the on_connect event is triggered on connection to an user stream
    # https://dev.twitter.com/streaming/overview/messages-types#friends-lists-friends
    @events.on_connect.handler
    def connection(self, data):
        consumer = asyncio.ensure_future(consume(queue))
        print("Connected to stream!")

    # the on_tweet event is triggered when a tweet seems to be sent on
    # the stream, by default retweets are included
    @events.on_tweet.handler
    async def tweet(self, data):
        await queue.put(data)
        #print("producing")


if __name__ == '__main__':
    queue = asyncio.Queue()
    client = Client(consumer_key=consumer_key,
                     consumer_secret=consumer_secret,
                     access_token=access_token,
                     access_token_secret=access_token_secret)
    client.run()

In [4]:
import csv

In [6]:
k = 3
counter = 0
error = 0
val = ''


def spacesaving():
    tabbb = [[counter, error, 'a'], 
             [counter, error, 'b'],
             [counter, error, 'c']]

    f = open("sample_reservoir.csv")
    csvreader = csv.reader(f)
    header = []
    header = next(csvreader)
    print(header)
    rows = []
    for row in csvreader:
        rows.append(row)
    print(rows)
    f.close()
    

    """# tableau des clés
    tab_keys = []
    for key in tabbb.items():
        tab_keys.append(key[0])
    print(tab_keys)"""

    # compare les clés avec les vals du cs"ArithmeticError
    '''
    i = 0
    while i < len(rows):
        print(i)
        j = 0
        while j < len(tab_keys):
            if tab_keys[j] == 'a':
                print('letsgoooo')
                j = j + 1
            else:
                print('incorrecte')
                j = j +1
        print('pololo')
        i = i + 1
    '''

    #le plus récent des loop
    '''i = 0
    print(len(rows))
    while i < len(rows):
        j = 0
        print('yo')
        while j < len(tabbb):
            if tabbb[j][2] == rows[i]:
                tabbb[j] = counter + 1
                j = j + 1
            else:
                print('mince')
                j = j + 1
        i = i + 1
        return tabbb'''
    
    '''elif tabbb[j][2] != rows[i][0] and j > 3:
        print('a')'''

    i=0
    while i < 13:
        j = 0
        k = 0
        while j < 3:
            if tabbb[j][2] == rows[i][0]:
                tabbb[j][0] = tabbb[j][0] + 1
                j = j +1
            else : 
                k = k + 1
                if k == 3:
                    print('yes')
                    tabbb[j][2] = rows[i][0]
                    tabbb[j][1] = tabbb[j][0]
                    tabbb[j][0] = tabbb[j][0] + 1
                j = j +1
        i = i +1
    print(tabbb)

spacesaving()

['start']
[['a'], ['b'], ['c'], ['a'], ['c'], ['c'], ['b'], ['b'], ['a'], ['b'], ['d'], ['c'], ['b']]
yes
yes
[[3, 0, 'a'], [5, 0, 'b'], [5, 4, 'c']]


In [5]:
counter = 0
error = 0
val = ''

#CE PROG DOIT ETRE BON JUSTE A TESTER

tabbb = [[counter, error, 'a'], [counter, error, 'b'], [counter, error, 'c']]

i=0
while i < 5:
    j = 0
    while j < 3:
        if tabbb[j][2] == a[i]:
            print('bon')
            tabbb[j][0] = tabbb[j][0] + 1
            j = j +1
        else : 
            print('dommage')
            j = j +1
    i = i +1
print(tabbb)



"""
tab_keys = []
for key in tab.items():
    tab_keys.append(key[0])

print(tab_keys)
len(tab_keys)

i = 0
while i < 1:
    print('allo')
    i = i+1
"""

bon
dommage
dommage
dommage
bon
dommage
dommage
dommage
bon
bon
dommage
dommage
bon
dommage
dommage
[[3, 0, 'a'], [1, 0, 'b'], [1, 0, 'c']]


"\ntab_keys = []\nfor key in tab.items():\n    tab_keys.append(key[0])\n\nprint(tab_keys)\nlen(tab_keys)\n\ni = 0\nwhile i < 1:\n    print('allo')\n    i = i+1\n"

In [17]:
counter = 0
error = 0


a = ['a', 'b', 'c', 'a', 'c', 'c','b','b','a','b','d','c','b']


def spacesaving():
    tabbb = [[counter, error, 'a'], 
             [counter, error, 'b'],
             [counter, error, 'c']]
    '''
    f = open("sample_reservoir.csv")
    csvreader = csv.reader(f)
    header = []
    header = next(csvreader)
    print(header)
    rows = []
    for row in csvreader:
        rows.append(row)
    print(rows)
    f.close()
    '''

    i=0
    while i < len(a):
        j = 0
        k = 0
        while j < 3:
            if tabbb[j][2] == a[i]:
                'print(j)'
                tabbb[j][0] = tabbb[j][0] + 1
                j = j +1
            else : 
                "print('non')"
                k = k + 1
                if k == 3:
                    print(a[i])
                    print(tabbb[j][2])
                    tabbb[j][2] = a[i]
                    tabbb[j][1] = tabbb[j][0]
                    tabbb[j][0] = tabbb[j][0] + 1
                j = j +1
        i = i +1
    
    print(tabbb)

spacesaving()

d
c
c
d
[[3, 0, 'a'], [5, 0, 'b'], [5, 4, 'c']]


In [27]:
'''oui = [[3, 0, 'a'], 
       [2, 0, 'b'],
       [5, 0, 'c'],
       [5, 0, 'e'],
       [4, 0, 'f'],
       [1, 0, 'g']]


k = 0
print(1 - (k - 1))
print(oui[len(oui) + (k - 1)][0])
print(oui[len(oui) + (k-2)][0])

k = 1
while k < len(oui):
    if oui[len(oui) + (k - 1)][0] > oui[len(oui) + (k-2)][0]:
        a = oui[len(oui) + (k-2)]
        oui[len(oui) + (k-2)] = oui[len(oui) + (k-1)]
        oui[len(oui) + (k-1)] = a
        k = k + 1
    else:
        k = k + 1'''


oui = [[3, 0, 'a'], 
       [2, 0, 'b'],
       [5, 0, 'c'],
       [5, 0, 'e'],
       [4, 0, 'f'],
       [1, 0, 'g']]


k = 2
print(len(oui))
print(1 - (k - 1))
'''print(oui[len(oui) + (k - 1)][0])
print(oui[len(oui) + (k-2)][0])
print(oui[len(oui)] + (k - 1))'''
prk = len(oui) - 1
print(oui[prk + (k - 2)][0])

k = 0
prk = len(oui) - 1
while k < prk:
    j = 0
    while j < prk:
        
        if oui[prk - (j)][0] > oui[prk - (j+1)][0]:
            a = oui[prk - (j+1)]
            oui[prk - (j+1)] = oui[prk - (j)]
            oui[prk - (j)] = a
            j = j + 1
        else:
            j = j + 1
    k = k + 1 
print(oui)


6
0
1
[[5, 0, 'c'], [5, 0, 'e'], [4, 0, 'f'], [3, 0, 'a'], [2, 0, 'b'], [1, 0, 'g']]


In [40]:
k = 3
counter = 0
epsilon = 0
dc = {}

a = ['a', 'b', 'c', 'a', 'c', 'c','b','b','a','b','d','c','b']
i = 0
print(a[i])


i = 0
l = 0



'''print(dc.get('a', 'caca'))

while i < len(a):
    if a[i] in dc.keys():
        print('yo')
        dc[a[i]][0] =  dc[a[i]][0] + 1
        i = i + 1
    elif l <= k:
        dc[a[i]] = (0, 0, a[i])
        l = l + 1
        i = i + 1
    else: 
        i = i + 1
print(dc)'''

while i < len(a):
    if a[i] not in dc.keys() and l < k:
        dc[a[i]] = (0, 0, a[i])
        min = dc[a[i]]
        l = l +1
        i = i + 1
    elif a[i] in dc.keys():
        demo_list = list(dc[a[i]])
        demo_list[0] += 1
        dc[a[i]] = tuple(demo_list)
        min = dc[a[i]] # il prend le b en tant que min alors que c le c le plus petit
        i = i + 1
    else:
        dc[a[i]] = (min[0] + 1, min[0], a[i])

        #faire le epsilon

        del dc[min[2]]
        min = dc[a[i]]
        print(min)
        i = i + 1

print(dc)


a
(4, 3, 'd')
(4, 3, 'b')
{'a': (2, 0, 'a'), 'd': (4, 3, 'd'), 'b': (4, 3, 'b')}


In [9]:
dico = {'a': [1321, 0, 'a'], 'b': [0, 0, 'b'], 'c': [0, 0, 'c']}
i = 0
a = ['a', 'b', 'c', 'a', 'c', 'c','b','b','a','b','d','c','b']
print(dico[a[i]])

'''if a[i] in dc.keys():
    print('yo')
    dc[a[i]][0] += 5
    print(dico)'''

[1321, 0, 'a']


"if a[i] in dc.keys():\n    print('yo')\n    dc[a[i]][0] += 5\n    print(dico)"

In [1]:
k = 3
counter = 0
epsilon = 0
dc = {}

a = ['a', 'b', 'c', 'a', 'c', 'c','b','b','a','b','d','c','b']
i = 0
print(a[i])


i = 0
l = 0

while i < len(a):
    if a[i] not in dc.keys() and l < k:
        dc[a[i]] = (0, 0, a[i])
        min = dc[a[i]]
        l = l +1
        i = i + 1
    elif a[i] in dc.keys():
        demo_list = list(dc[a[i]])
        demo_list[0] += 1
        dc[a[i]] = tuple(demo_list)
        #min = dc[a[i]] # il prend le b en tant que min alors que c le c le plus petit
        i = i + 1
    else:
        dc[a[i]] = (min[0] + 1, min[0], a[i])
        del dc[min[2]]
        min = dc[a[i]]
        print(min)
        i = i + 1

print(dc)

a
(1, 0, 'd')
(2, 1, 'c')
{'a': (2, 0, 'a'), 'b': (4, 0, 'b'), 'c': (2, 1, 'c')}
